# Simulation experiment using noisy data 

Run entire simulation experiment multiple times to generate confidence interval.  The simulation experiment can be found in ```functions/pipeline.py```

In [1]:
%load_ext autoreload
%autoreload 2

from joblib import Parallel, delayed
import multiprocessing
import sys
import os
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

sys.path.append("../../")
from functions import pipelines, utils

from numpy.random import seed
randomState = 123
seed(randomState)

Using TensorFlow backend.


In [ ]:
# Read in config variables
config_file = os.path.abspath(os.path.join(os.getcwd(),"../../configs", "config_Pa_experiment_combat.tsv"))
params = utils.read_config(config_file)

In [ ]:
# Load parameters
dataset_name = params["dataset_name"]
analysis_name = params["analysis_name"]
NN_architecture = params["NN_architecture"]
num_simulated_experiments = params["num_simulated_samples"]
lst_num_partitions = params["lst_num_experiments"]
use_pca = params["use_pca"]
num_PCs = params["num_PCs"]
local_dir = params["local_dir"]
correction_method = params["correction_method"]

iterations = params["iterations"] 
num_cores = params["num_cores"]

In [ ]:
# Additional parameters
file_prefix = "Partition"
corrected = False

In [3]:
# Input
base_dir = os.path.abspath(
      os.path.join(
          os.getcwd(), "../.."))

normalized_data_file = os.path.join(
    base_dir,
    dataset_name,    
    "data",
    "input",
    "train_set_normalized.pcl")

experiment_ids_file = os.path.join(
    base_dir,
    dataset_name,
    "data",
    "metadata",
    "experiment_ids.txt")

In [4]:
# Output files
similarity_uncorrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    dataset_name +"_experiment_lvl_sim_similarity_uncorrected_"+correction_method+".pickle")

ci_uncorrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    dataset_name +"_experiment_lvl_sim_ci_uncorrected_"+correction_method+".pickle")

similarity_permuted_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    dataset_name +"_experiment_lvl_sim_permuted")

In [5]:
# Run multiple simulations - uncorrected
results = Parallel(n_jobs=num_cores, verbose=100)(
    delayed(
        pipelines.experiment_level_simulation_uncorrected)(i,
                                                           NN_architecture,
                                                           dataset_name,
                                                           analysis_name,
                                                           num_simulated_experiments,
                                                           lst_num_partitions,
                                                           corrected,
                                                           use_pca,
                                                           num_PCs,
                                                           file_prefix,
                                                           normalized_data_file,
                                                           experiment_ids_file,
                                                           local_dir) for i in iterations)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed: 73.5min
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 75.5min remaining: 113.3min
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed: 76.3min remaining: 50.8min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 78.4min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 78.4min finished


In [6]:
# permuted score
permuted_score = results[0][0]

In [7]:
# Concatenate output dataframes
all_svcca_scores = pd.DataFrame()

for i in iterations:
    all_svcca_scores = pd.concat([all_svcca_scores, results[i][1]], axis=1)

all_svcca_scores

,score,score,score,score,score
number of partitions,,,,,
1,0.999993,0.999993,0.999992,0.999992,0.999992
2,0.902303,0.904220,0.913136,0.908107,0.907379
3,0.805412,0.812215,0.813478,0.807043,0.818391
5,0.624638,0.631114,0.624973,0.648582,0.636305
10,0.231828,0.243049,0.247360,0.235669,0.249036
20,0.321926,0.313169,0.314560,0.329857,0.332546
30,0.353129,0.351125,0.369350,0.394281,0.362437
50,0.386305,0.420258,0.431910,0.437215,0.412212
70,0.489549,0.484146,0.430870,0.521731,0.517814


In [8]:
# Get median for each row (number of experiments)
mean_scores = all_svcca_scores.mean(axis=1).to_frame()
mean_scores.columns = ['score']
mean_scores

,score
number of partitions,
1,0.999993
2,0.907029
3,0.811308
5,0.633122
10,0.241388
20,0.322412
30,0.366064
50,0.417580
70,0.488822


In [9]:
# Get standard dev for each row (number of experiments)
import math
std_scores = (all_svcca_scores.std(axis=1)/math.sqrt(10)).to_frame()
std_scores.columns = ['score']
std_scores

,score
number of partitions,
1,8.253284e-08
2,1.311543e-03
3,1.648093e-03
5,3.128692e-03
10,2.350616e-03
20,2.763431e-03
30,5.501461e-03
50,6.334439e-03
70,1.151551e-02


In [10]:
# Get confidence interval for each row (number of experiments)
err = std_scores*1.96

In [11]:
# Get boundaries of confidence interval
ymax = mean_scores + err
ymin = mean_scores - err

ci = pd.concat([ymin, ymax], axis=1)
ci.columns = ['ymin', 'ymax']
ci

,ymin,ymax
number of partitions,,
1,0.999992,0.999993
2,0.904459,0.909600
3,0.808078,0.814538
5,0.626990,0.639255
10,0.236781,0.245996
20,0.316995,0.327828
30,0.355281,0.376847
50,0.405165,0.429996
70,0.466251,0.511392


In [12]:
mean_scores

,score
number of partitions,
1,0.999993
2,0.907029
3,0.811308
5,0.633122
10,0.241388
20,0.322412
30,0.366064
50,0.417580
70,0.488822


In [13]:
# Pickle dataframe of mean scores scores for first run, interval
mean_scores.to_pickle(similarity_uncorrected_file)
ci.to_pickle(ci_uncorrected_file)
np.save(similarity_permuted_file, permuted_score)